<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">LLMをゼロから作成する</a>書籍の補足コード（<a href="https://sebastianraschka.com">Sebastian Raschka</a>著）<br>
<br>コードリポジトリ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# Llama 3とOllamaによる指示データセットの生成

- このノートブックは、ollama経由の80億パラメータのLlama 3モデルを使用して、「Magpie: Alignment Data Synthesis from Scratch by Prompting Aligned LLMs with Nothing」論文（[https://arxiv.org/abs/2406.08464](https://arxiv.org/abs/2406.08464)）で提案された「ハック」を使用して合成データセットを生成します

- 生成されるデータセットは、Alpacaにあるような「instruction」と「output」フィールドを持つ指示データセットになります：


```python
{
    "instruction": "What is the atomic number of helium?",
    "output": "The atomic number of helium is 2.",
},
```

- このコードはGPUを必要とせず、ラップトップで実行できます（M3 MacBook Airでテストされました）

*ここで作成される指示データセットは教育目的のものであることに注意してください。ただし、Meta AIのLlama 3の関連ライセンス契約の条件を遵守して使用することは利用者の責任です。*

In [1]:
from importlib.metadata import version

pkgs = [
    "tqdm",    # Progress bar
]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## OllamaのインストールとLlama 3のダウンロード

- OllamaはLLMを効率的に実行するアプリケーションです
- これは[llama.cpp](https://github.com/ggerganov/llama.cpp)のラッパーで、LLMを純粋なC/C++で実装して効率を最大化します
- これはLLMを使用してテキストを生成する（推論）ためのツールであり、LLMの訓練やファインチューニングのためのものではないことに注意してください
- 以下のコードを実行する前に、[https://ollama.com](https://ollama.com)にアクセスして指示に従ってollamaをインストールしてください（例：「Download」ボタンをクリックして、お使いのオペレーティングシステム用のollamaアプリケーションをダウンロード）

- macOSおよびWindowsユーザーの場合、ダウンロードしたollamaアプリケーションをクリックしてください。コマンドラインの使用をインストールするかを尋ねられた場合は「yes」と答えてください
- Linuxユーザーは、ollamaウェブサイトで提供されているインストールコマンドを使用できます

- 一般的に、コマンドラインからollamaを使用する前に、ollamaアプリケーションを開始するか、別のターミナルで`ollama serve`を実行する必要があります

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- ollamaアプリケーションまたは`ollama serve`が実行されている状態で、別のターミナルで、コマンドラインで以下のコマンドを実行して80億パラメータのLlama 3モデルを試してください（4.7 GBのストレージスペースを占めるモデルは、初回実行時に自動的にダウンロードされます）

```bash
# 8Bモデル
ollama run llama3
```


出力は以下のようになります：

```
$ ollama run llama3
pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
```

- `llama3`は指示ファインチューニングされた80億パラメータのLlama 3モデルを指すことに注意してください

- 代わりに、お使いのマシンがサポートしている場合、`llama3`を`llama3:70b`に置き換えることで、より大きな700億パラメータのLlama 3モデルを使用することもできます

- ダウンロードが完了すると、モデルとチャットできるコマンドラインプロンプトが表示されます

- 「What do llamas eat?」のようなプロンプトを試してみてください。以下のような出力が返されるはずです：

```
>>> What do llamas eat?
Llamas are ruminant animals, which means they have a four-chambered 
stomach and eat plants that are high in fiber. In the wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, and barley.
```

- `/bye`と入力してこのセッションを終了できます

## OllamaのREST APIの使用

- さて、モデルと対話する別の方法として、以下の関数を使ってPythonでREST APIを使用する方法があります
- このノートブックの次のセルを実行する前に、上記で説明したように以下のいずれかでollamaが実行されていることを確認してください：
  - ターミナルで`ollama serve`
  - ollamaアプリケーション
- 次に、以下のコードセルを実行してモデルにクエリを送信してください

- まず、APIが意図した通りに動作することを確認するために、簡単な例でAPIを試してみましょう：

In [2]:
import urllib.request
import json

def query_model(prompt, model="llama3", url="http://localhost:11434/api/chat", role="user"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "seed": 123,        # for deterministic responses
        "temperature": 1.,   # for deterministic responses
        "top_p": 1,         
        "messages": [
            {"role": role, "content": prompt}
        ]
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

In [3]:
result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and even weeds.
2. Hay: They enjoy eating hay, such as alfalfa or timothy hay, which provides them with fiber, protein, and other essential nutrients.
3. Grains: Llamas may eat grains like oats, barley, or corn as a supplement to their diet.
4. Leaves: They will also munch on leaves from trees and shrubs, including clover, alfalfa, and various types of leaves.
5. Fruits and vegetables: In the wild, llamas might eat fruits and vegetables that grow in their natural habitat, such as apples, carrots, or potatoes.

In general, a llama's diet should consist of:

* 50% grasses and hay
* 20% grains (like oats or corn)
* 10% leaves and other plant material
* 5% fruits and vegetables (as treats)

It's essential to provide llamas with a balanced diet that meets their nutritional n

## 指示の抽出

- では、論文で提案された「ハック」を使用してみましょう：空のプロンプトテンプレート`"<|begin_of_text|><|start_header_id|>user<|end_header_id|>"`を提供すると、指示ファインチューニングされたLlama 3モデルが指示を生成します

In [4]:
def extract_instruction(text):
    for content in text.split("\n"):
        if content:
            return content.strip()

In [5]:
query = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"

result = query_model(query, role="assistant")
instruction = extract_instruction(result)
print(instruction)

I am trying to find a way to make my child's birthday party more special and unique. What are some creative ideas you have?


- 上記でわかるように、驚くべきことに、モデルは実際に指示を生成しました

## 応答の生成

- 次のステップは、対応する応答を作成することです。これは単に指示を入力として渡すことで行うことができます

In [6]:
response = query_model(instruction, role="user")
print(response)

What an exciting question! I'd be delighted to help you come up with some creative and unique ideas to make your child's birthday party truly special!

Here are a few ideas to get you started:

1. **Themed Scavenger Hunt**: Plan a scavenger hunt based on the birthday child's favorite theme (e.g., superheroes, animals, or princesses). Hide clues and challenges throughout the party area, leading up to a final surprise.
2. **DIY Crafts Station**: Set up a craft station where kids can create their own party favors, such as customized t-shirts, crowns, or jewelry. This activity encourages creativity and makes for a memorable keepsake.
3. **Mystery Box Challenge**: Fill mystery boxes with different textures, smells, and sounds. Have the kids guess what's inside each box without looking. This game promotes problem-solving and teamwork.
4. **Indoor Camping Adventure**: Set up a cozy indoor "camping" area with sleeping bags, flashlights, and s'mores-making stations. Kids can enjoy a camping exp

## データセットの生成

- このアプローチを任意の数のデータサンプルまでスケールアップできます（長さや品質によるオプションのフィルタリング（例：別のLLMを使用して生成されたデータを評価する）を適用することをお勧めします）
- 以下では、5つの合成指示-応答ペアを生成します。これはM3 MacBook Airで約3分かかります
- （指示ファインチューニングに適したデータセットを生成するには、これを少なくとも1kから50kに増やし、より短時間で例を生成するためにGPUで実行することをお勧めします）

**ヒント**

- `model="llama3"`を`model="llama3:70b"`に変更することで、さらに高品質な応答を生成できますが、これにはより多くの計算リソースが必要になります

In [7]:
from tqdm import tqdm

dataset_size = 5
dataset = []

for i in tqdm(range(dataset_size)):

    result = query_model(query, role="assistant")
    instruction = extract_instruction(result)
    response = query_model(instruction, role="user")
    entry = {
        "instruction": instruction,
        "output": response
    }
    dataset.append(entry)

100%|█████████████████████████████████████████████| 5/5 [02:37<00:00, 31.41s/it]


In [8]:
with open("instruction-data-llama3-7b.json", "w") as file:
    json.dump(dataset, file, indent=4)

In [9]:
!cat instruction-data-llama3-7b.json

[
    {
        "instruction": "What is the significance of the number 7 in various cultures and religions?",
        "output": "The number 7 has been a significant and recurring theme across many cultures and religions, often imbuing it with special meaning and symbolism. Here are some examples:\n\n1. **Numerology**: In numerology, the number 7 is considered sacred and mystical, associated with spiritual awakening, introspection, and enlightenment.\n2. **Judaism**: The Torah has seven days of creation, seven weeks in the wilderness, and seven years of rest (Sabbatical year). Seven is also a symbol of completion or perfection.\n3. **Christianity**: In Christianity, there are seven deadly sins, seven virtues, and seven sacraments. Jesus was said to have spoken seven sermons, and the number 7 appears in various biblical accounts, such as the seven days of creation and the seven angels who appear before God.\n4. **Islam**: In Islamic tradition, there are seven heavens, seven earths, and s